# Library importing

In [ ]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer

# data pre-proccessing

## data reading

In [ ]:
data = pd.read_csv("/kaggle/input/depi-dataset/data.csv")

In [ ]:
data.head(3)

In [ ]:
final_data = pd.read_csv("/kaggle/input/data-cleaning/preprocessing")

## feature engineering

In [ ]:
data['date'] = pd.to_datetime(data['date'])
data['year'] = data['date'].dt.year
data['month'] = data['date'].dt.month
data['day'] = data['date'].dt.day

In [ ]:
data['price']= data['price_in_dollars']/data['sales']
data.fillna(0,inplace=True)

In [ ]:
data['wm_yr_wk']=data['wm_yr_wk']%100

In [ ]:
data.drop(columns = ['d'],inplace = True)
data.drop(columns=['date'],inplace = True)
data.drop(columns = ['price_in_dollars'],inplace = True)

## Encoding

In [ ]:
categorical_cols = ['item_id','store_id','event_name_1','event_name_2','event_type_1','event_type_2']
numeric_cols = ["wm_yr_wk", "wday", "snap","year", "month", "day", "price"]

In [ ]:
categorical_transformer = Pipeline(steps=[
    ("encoder", OrdinalEncoder(handle_unknown="use_encoded_value")),
    ("scaler", MinMaxScaler())
])
numeric_transformer = "passthrough"
preprocessor = ColumnTransformer(transformers=[
    ("cat", categorical_transformer, categorical_cols),
    ("num", numeric_transformer, numeric_cols)
])

In [ ]:
preprocessor

## Data spliting

In [ ]:
y=data['sales']
x=data.drop('sales',axis = 1)

# Model